In [1]:
import pandas as pd
import numpy as np

from babel.numbers import decimal, format_decimal
import utils
from blupy.subject.basic_client_infos import basic_client_infos as clients
from blupy.connections.db_connections import query_in_db as query_in_db

In [2]:
def churn(c):
    
    if c['time_in_churn'] > 30:
        c = 1
    
    else:
        c = 0
        
    return c


def early_churn(c):
    
    if c['time_in_blu_days'] <= 60 & c['time_in_churn'] > 30 :
        c = 1
    
    else:
        c = 0
        
    return c


def basic_client_infos(store_id = (12,80,115), department_id = (6,7,8,9,45,36)):

    query = """select c1.client_id as client_id,
    min(ct.happened_at) first_buy,
    max(ct.happened_at) last_buy,
    sum(gross_value) total_buy,
    count(distinct(client_receiver_id)) n_forns,
    count(c1.client_id) as n_buys
    from client_transactions ct
    inner join clients c1 on c1.id = ct.client_id
    inner join clients c2 on c2.id = ct.client_receiver_id
    where ct.transaction_category_id = 33
    and status = 'confirmed'
    and c1.store_id in {stores}
    and c1.department_id in {departaments}
    and c2.distributor = True
    and c2.seller = False
    and gross_value < -10
    and c1.distributor = False
    and c1.seller = False
    and nature = 'outflow'
    group by c1.client_id""".format(departaments = department_id,stores = store_id)
    buys = query_in_db(query, 'pagnet_read_replica.json')    


    query = """select c1.client_id as client_id, 
    min(ct.happened_at) first_buy,
    max(ct.happened_at) last_buy,
    sum(gross_value) total_buy,
    count(distinct(client_receiver_id)) n_forns,
    count(c1.client_id) as n_buys

    from client_transactions ct

    inner join clients c1 on c1.id = ct.client_id
    inner join clients c2 on c2.id = ct.client_receiver_id


    where ct.transaction_category_id = 33
    and status = 'confirmed'
    and c1.store_id in {stores}
    and c1.department_id in {departaments}
    and c2.distributor = True
    and c2.seller = False
    and c1.distributor = False
    and c1.seller = False
    and nature = 'outflow'

    group by c1.client_id""".format(departaments = department_id,stores = store_id)

    buys = query_in_db(query, 'pagnet_read_replica.json')     
    
    base = base.merge(buys, on = 'client_id', how = 'left')
    
                       
    query = """select c1.client_id  as client_id, 
    date_trunc('days',min(ct.happened_at)) first_true_sell

    from client_transactions ct

    inner join clients c1 on c1.id = ct.client_id

    where ct.transaction_category_id in (92,93)
    and status = 'confirmed'
    and c1.distributor = False
    and c1.seller = False
    and c1.store_id in {stores}
    and c1.department_id in {departaments}
    and gross_value >= 10


    group by c1.client_id""".format(departaments = department_id,stores = store_id)
    first_true_sell = query_in_db(query, 'pagnet_read_replica.json')            
                       
    base = base.merge(first_true_sell, on = 'client_id', how = 'left')
        
    base['time_in_churn'] =  (pd.Timestamp.now() - base.last_sell).dt.days
    base['time_in_blu_months'] =  np.rint((base.last_sell - base.first_true_sell ).dt.days/30)
    base['time_in_blu_days'] =  np.rint((base.last_sell - base.first_true_sell ).dt.days)
    
    got_loyalty= loyalty(store_id, department_id )
    
    base=base.merge(got_loyalty, on = 'client_id', how = 'left')
    
    base['time_to_loyal']= (base.loyal_date - base.first_sell).dt.days
    base['churn']=base.apply(churn, axis= 1)
    base['earl_churn']=base.apply(early_churn, axis = 1)
    
    pipe_activate=checking_activate()
    base = base.merge(pipe_activate, on = 'client_id', how = 'left')
    
    return base



def loyalty(store_id = (12,80,115), department_id = (6,7,8,9,45,36)):  

    query = """select clients.client_id as client_id,  date_trunc('days', happened_at) date, sum(gross_value) gross_value from client_transactions

    inner join clients on clients.id = client_transactions.client_id


    where transaction_category_id in (92,93)
    and status = 'confirmed'
    and clients.department_id in {departaments}
    and clients.store_id in {stores}

    group by clients.client_id,  date_trunc('days', happened_at)""".format(stores = store_id, departaments = department_id)

    ganho = query_in_db(query, 'pagnet_read_replica.json')

    ganho=ganho.sort_values(by = ['client_id','date']).groupby(by=['client_id','date']).sum().groupby(level=[0]).cumsum().reset_index()
    ganho=ganho[ganho.gross_value > 500].sort_values('date').drop_duplicates('client_id')
    ganho.rename(columns= {'date':'loyal_date'}, inplace = True)
    
    return ganho[['client_id','loyal_date']]

def checking_activate():


    query = """select data_additional_data_old_value_formatted as come_from,
    data_additional_data_new_value_formatted as go_to,
    data_log_time as ativated_at,
    owner_name as owner_name,
    pipedeals.cnpj

    from pipedeal_changes


    inner join pipedeals on pipedeal_changes.data_item_id = pipedeals.id


    where data_new_value in (490,372)
    and data_old_value = 378 """
    ativacao = query_in_db(query, 'blu_datascience.json')
    
    
    query = """select client_id, cpf_cnpj  cnpj from clients """
    clients = query_in_db(query, 'pagnet_read_replica.json')

    
    ativacao = ativacao.merge(clients, on = ['cnpj'])
    
    
    ativacao = ativacao.sort_values(by = 'ativated_at').drop_duplicates('client_id')
    ativacao = ativacao[['client_id','ativated_at']]
    
    
    return ativacao

In [3]:
base = basic_client_infos(store_id = (12,80,115,149), department_id = (9,45,36,25,5,6,7,8,9,46,47,149))

In [4]:
#base = clients(store_id = (12,80,115,149), department_id = (9,45,36,25,5,6,7,8,9,46,47,149))
base = base[base.client_id != 25997]
base = base[base.client_id != 38983]
base.to_csv('base.csv', index = False)

In [5]:
#pd.read_csv('./data/base.csv').columns #11346 rows × 18 columns / 11429 rows × 19 columns